# 50-异步Web框架

## 📚 用途说明

**学习目标**：
- 掌握FastAPI异步Web框架的核心概念和使用方法
- 学会创建异步路由处理器和依赖注入系统
- 理解异步中间件和后台任务处理机制
- 能够构建高性能的异步Web应用系统

**前置要求**：
- 已完成异步网络编程学习
- 熟练掌握异步HTTP客户端和WebSocket技术
- 了解基本的Web开发概念和RESTful API设计
- 理解依赖注入和中间件模式

**与LangChain关联**：
- FastAPI异步路由优化LangChain的API响应性能
- 依赖注入模式支持LangChain的组件组合
- 后台任务处理LangChain的长时间推理过程
- 异步Web框架增强LangChain的服务化能力

---

## 🔢 知识点覆盖

### 6.5 异步Web框架 [⭐⭐进阶]
**知识点说明**：异步Web框架是构建高性能API服务的核心技术，对于LangChain的服务化部署和实时处理至关重要。

**学习要求**：
- 掌握FastAPI异步Web框架的核心概念和使用方法
- 学会创建异步路由处理器和依赖注入系统
- 理解异步中间件和后台任务处理机制
- 能够构建高性能的异步Web应用系统

**案例要求**：
- 实现完整的异步FastAPI应用
- 进行异步Web性能对比分析
- 应用异步Web技术解决实际问题
- 验证点：能独立构建高性能异步Web应用

In [ ]:
print("🌐 异步Web框架:")
print("=" * 50)

# 导入必要的库
import asyncio
import time
import datetime
import json
import random
import uuid
from typing import List, Dict, Any, Optional, Union
from dataclasses import dataclass, asdict
from contextlib import asynccontextmanager

# FastAPI相关导入
from fastapi import FastAPI, HTTPException, Depends, BackgroundTasks, Request, Response
from fastapi.middleware.cors import CORSMiddleware
from fastapi.middleware.gzip import GZipMiddleware
from fastapi.responses import JSONResponse, StreamingResponse
from fastapi.security import HTTPBearer, HTTPAuthorizationCredentials
from pydantic import BaseModel, Field, validator
import uvicorn
import nest_asyncio

# 允许在Jupyter中运行异步代码
nest_asyncio.apply()

print(f"✅ Python版本: {__import__('sys').version}")
print(f"✅ FastAPI版本: {__import__('fastapi').__version__}")
print(f"✅ Uvicorn版本: {__import__('uvicorn').__version__}")
print(f"✅ 当前时间: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# 1. FastAPI基础概念
print(f"📝 1. FastAPI基础概念:")

# 1.1 Pydantic模型定义
print(f"\n   🔍 1.1 Pydantic模型定义:")

class UserCreate(BaseModel):
    """用户创建模型"""
    username: str = Field(..., min_length=3, max_length=50)
    email: str = Field(..., regex=r'^[^@]+@[^@]+\.[^@]+$')
    age: int = Field(..., ge=18, le=120)
    
    @validator('username')
    def username_alphanumeric(cls, v):
        if not v.isalnum():
            raise ValueError('用户名必须为字母数字')
        return v

class UserResponse(BaseModel):
    """用户响应模型"""
    id: int
    username: str
    email: str
    age: int
    created_at: datetime.datetime
    
    class Config:
        from_attributes = True

class MessageRequest(BaseModel):
    """消息请求模型"""
    content: str = Field(..., min_length=1, max_length=1000)
    user_id: int
    priority: str = Field(default='normal', regex='^(low|normal|high)$')

class MessageResponse(BaseModel):
    """消息响应模型"""
    id: int
    content: str
    user_id: int
    priority: str
    created_at: datetime.datetime
    processed: bool = False

class LLMRequest(BaseModel):
    """LLM请求模型（LangChain相关）"""
    prompt: str = Field(..., min_length=1, max_length=2000)
    model: str = Field(default='gpt-3.5-turbo')
    temperature: float = Field(default=0.7, ge=0.0, le=2.0)
    max_tokens: int = Field(default=100, ge=1, le=1000)
    stream: bool = Field(default=False)

class LLMResponse(BaseModel):
    """LLM响应模型"""
    response: str
    model: str
    tokens_used: int
    processing_time: float

# 模拟数据存储
users_db = {}
messages_db = {}
user_counter = 1
message_counter = 1

print(f"   ✅ Pydantic模型定义完成")
print(f"      - UserCreate: 用户创建验证模型")
print(f"      - UserResponse: 用户响应模型")
print(f"      - MessageRequest: 消息请求模型")
print(f"      - MessageResponse: 消息响应模型")
print(f"      - LLMRequest: LLM请求模型（LangChain集成）")
print(f"      - LLMResponse: LLM响应模型")

# 1.2 依赖注入系统
print(f"\n   🔧 1.2 依赖注入系统:")

# 数据库依赖
async def get_database():
    """模拟数据库连接依赖"""
    print(f"      📊 获取数据库连接")
    await asyncio.sleep(0.01)  # 模拟连接延迟
    return {"connection": "mock_db", "status": "connected"}

# 认证依赖
security = HTTPBearer()

async def get_current_user(credentials: HTTPAuthorizationCredentials = Depends(security)):
    """获取当前用户依赖"""
    token = credentials.credentials
    
    # 模拟token验证
    if token == "valid_token_123":
        return {"user_id": 1, "username": "test_user", "role": "user"}
    elif token == "admin_token_456":
        return {"user_id": 2, "username": "admin", "role": "admin"}
    else:
        raise HTTPException(status_code=401, detail="无效的认证令牌")

# 管理员权限依赖
async def get_admin_user(current_user: dict = Depends(get_current_user)):
    """管理员权限依赖"""
    if current_user.get("role") != "admin":
        raise HTTPException(status_code=403, detail="需要管理员权限")
    return current_user

# 限流依赖
class RateLimiter:
    """限流器"""
    
    def __init__(self, max_requests: int = 10, window_seconds: int = 60):
        self.max_requests = max_requests
        self.window_seconds = window_seconds
        self.requests = {}
    
    async def check_rate_limit(self, request: Request):
        """检查限流"""
        client_ip = request.client.host
        current_time = time.time()
        
        # 清理过期请求
        if client_ip in self.requests:
            self.requests[client_ip] = [
                req_time for req_time in self.requests[client_ip]
                if current_time - req_time < self.window_seconds
            ]
        else:
            self.requests[client_ip] = []
        
        # 检查是否超过限制
        if len(self.requests[client_ip]) >= self.max_requests:
            raise HTTPException(
                status_code=429, 
                detail=f"请求频率过高，每{self.window_seconds}秒最多{self.max_requests}次请求"
            )
        
        # 记录当前请求
        self.requests[client_ip].append(current_time)
        
        return {"allowed": True, "remaining": self.max_requests - len(self.requests[client_ip])}

rate_limiter = RateLimiter(max_requests=5, window_seconds=60)

print(f"   ✅ 依赖注入系统完成")
print(f"      - get_database: 数据库连接依赖")
print(f"      - get_current_user: 用户认证依赖")
print(f"      - get_admin_user: 管理员权限依赖")
print(f"      - RateLimiter: 限流依赖")

# 1.3 异步路由处理器
print(f"\n   🛣️ 1.3 异步路由处理器:")

# 创建FastAPI应用
app = FastAPI(
    title="异步Web框架演示",
    description="FastAPI异步Web框架完整演示",
    version="1.0.0",
    docs_url="/docs",
    redoc_url="/redoc"
)

# 添加中间件
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

app.add_middleware(GZipMiddleware, minimum_size=1000)

# 全局变量存储性能统计
performance_stats = {
    "total_requests": 0,
    "total_response_time": 0.0,
    "error_count": 0,
    "endpoint_stats": {}
}

@app.middleware("http")
async def performance_middleware(request: Request, call_next):
    """性能监控中间件"""
    start_time = time.time()
    
    # 处理请求
    response = await call_next(request)
    
    # 记录性能数据
    process_time = time.time() - start_time
    
    performance_stats["total_requests"] += 1
    performance_stats["total_response_time"] += process_time
    
    if response.status_code >= 400:
        performance_stats["error_count"] += 1
    
    endpoint = request.url.path
    if endpoint not in performance_stats["endpoint_stats"]:
        performance_stats["endpoint_stats"][endpoint] = {
            "requests": 0,
            "total_time": 0.0,
            "errors": 0
        }
    
    performance_stats["endpoint_stats"][endpoint]["requests"] += 1
    performance_stats["endpoint_stats"][endpoint]["total_time"] += process_time
    
    if response.status_code >= 400:
        performance_stats["endpoint_stats"][endpoint]["errors"] += 1
    
    response.headers["X-Process-Time"] = str(process_time)
    
    return response

# 基础路由
@app.get("/")
async def root():
    """根路由"""
    return {
        "message": "异步Web框架演示API",
        "version": "1.0.0",
        "timestamp": datetime.datetime.now().isoformat(),
        "endpoints": {
            "users": "/users",
            "messages": "/messages",
            "llm": "/llm",
            "stats": "/stats",
            "docs": "/docs"
        }
    }

# 用户管理路由
@app.post("/users", response_model=UserResponse)
async def create_user(
    user: UserCreate, 
    db: dict = Depends(get_database),
    rate_limit: dict = Depends(rate_limiter.check_rate_limit)
):
    """创建用户"""
    global user_counter
    
    # 模拟数据库操作
    await asyncio.sleep(0.01)
    
    # 检查用户名是否已存在
    for existing_user in users_db.values():
        if existing_user["username"] == user.username:
            raise HTTPException(status_code=400, detail="用户名已存在")
        if existing_user["email"] == user.email:
            raise HTTPException(status_code=400, detail="邮箱已存在")
    
    # 创建新用户
    new_user = {
        "id": user_counter,
        "username": user.username,
        "email": user.email,
        "age": user.age,
        "created_at": datetime.datetime.now()
    }
    
    users_db[user_counter] = new_user
    user_counter += 1
    
    return new_user

@app.get("/users/{user_id}", response_model=UserResponse)
async def get_user(user_id: int):
    """获取用户"""
    await asyncio.sleep(0.01)  # 模拟数据库查询
    
    if user_id not in users_db:
        raise HTTPException(status_code=404, detail="用户不存在")
    
    return users_db[user_id]

@app.get("/users", response_model=List[UserResponse])
async def list_users(
    skip: int = 0,
    limit: int = 10,
    current_user: dict = Depends(get_current_user)
):
    """获取用户列表"""
    await asyncio.sleep(0.02)  # 模拟数据库查询
    
    users = list(users_db.values())[skip:skip + limit]
    return users

# 消息处理路由
@app.post("/messages", response_model=MessageResponse)
async def create_message(
    message: MessageRequest,
    background_tasks: BackgroundTasks,
    db: dict = Depends(get_database)
):
    """创建消息"""
    global message_counter
    
    # 验证用户是否存在
    if message.user_id not in users_db:
        raise HTTPException(status_code=404, detail="用户不存在")
    
    # 创建消息
    new_message = {
        "id": message_counter,
        "content": message.content,
        "user_id": message.user_id,
        "priority": message.priority,
        "created_at": datetime.datetime.now(),
        "processed": False
    }
    
    messages_db[message_counter] = new_message
    message_counter += 1
    
    # 添加后台任务处理消息
    background_tasks.add_task(process_message_async, new_message["id"])
    
    return new_message

@app.get("/messages/{message_id}", response_model=MessageResponse)
async def get_message(message_id: int):
    """获取消息"""
    await asyncio.sleep(0.01)
    
    if message_id not in messages_db:
        raise HTTPException(status_code=404, detail="消息不存在")
    
    return messages_db[message_id]

# LLM处理路由（LangChain集成）
@app.post("/llm", response_model=LLMResponse)
async def process_llm_request(
    request: LLMRequest,
    current_user: dict = Depends(get_current_user)
):
    """处理LLM请求"""
    start_time = time.time()
    
    # 模拟LLM处理（实际应用中会调用LangChain）
    processing_time = random.uniform(0.5, 2.0)
    await asyncio.sleep(processing_time)
    
    # 模拟LLM响应
    response_text = f"这是对'{request.prompt[:50]}...'的模拟响应。使用的模型是{request.model}，温度设置为{request.temperature}。"
    tokens_used = len(response_text.split()) + len(request.prompt.split())
    
    total_time = time.time() - start_time
    
    return LLMResponse(
        response=response_text,
        model=request.model,
        tokens_used=tokens_used,
        processing_time=total_time
    )

@app.post("/llm/stream")
async def stream_llm_response(
    request: LLMRequest,
    current_user: dict = Depends(get_current_user)
):
    """流式LLM响应"""
    
    async def generate_stream():
        """生成流式响应"""
        response_text = f"这是对'{request.prompt[:50]}...'的流式响应。"
        words = response_text.split()
        
        for i, word in enumerate(words):
            chunk = {
                "id": i,
                "word": word,
                "finished": i == len(words) - 1
            }
            yield f"data: {json.dumps(chunk, ensure_ascii=False)}\n\n"
            await asyncio.sleep(0.1)  # 模拟流式延迟
    
    return StreamingResponse(
        generate_stream(),
        media_type="text/plain",
        headers={"Cache-Control": "no-cache", "Connection": "keep-alive"}
    )

# 管理员路由
@app.get("/admin/stats")
async def get_admin_stats(admin_user: dict = Depends(get_admin_user)):
    """获取管理员统计信息"""
    return {
        "performance": performance_stats,
        "users_count": len(users_db),
        "messages_count": len(messages_db),
        "admin_info": admin_user
    }

@app.delete("/admin/messages/{message_id}")
async def delete_message_admin(
    message_id: int,
    admin_user: dict = Depends(get_admin_user)
):
    """管理员删除消息"""
    if message_id not in messages_db:
        raise HTTPException(status_code=404, detail="消息不存在")
    
    del messages_db[message_id]
    return {"message": "消息已删除", "deleted_id": message_id}

# 后台任务函数
async def process_message_async(message_id: int):
    """异步处理消息"""
    await asyncio.sleep(random.uniform(1, 3))  # 模拟处理时间
    
    if message_id in messages_db:
        messages_db[message_id]["processed"] = True
        print(f"      ✅ 消息 {message_id} 处理完成")

print(f"   ✅ 异步路由处理器完成")
print(f"      - 根路由: /")
print(f"      - 用户管理: /users")
print(f"      - 消息处理: /messages")
print(f"      - LLM处理: /llm")
print(f"      - 管理员功能: /admin")
print(f"      - 性能监控中间件已启用")

print(f"\n✅ FastAPI基础概念完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握FastAPI异步Web框架的核心概念和使用方法")
print(f"   ✓ 学会创建异步路由处理器和依赖注入系统")
print(f"   ✓ 理解Pydantic模型验证和中间件机制")
print(f"   ✓ 能够构建基础的异步Web应用系统")

### 异步中间件和后台任务 [⭐⭐进阶]
**知识点说明**：异步中间件和后台任务是构建高性能Web应用的关键组件，对于LangChain的长时间推理和实时处理非常重要。

**学习要求**：
- 掌握异步中间件的开发和使用方法
- 理解后台任务的异步处理机制
- 学会请求生命周期管理和错误处理
- 能够实现高效的异步中间件系统

**案例要求**：
- 实现完整的异步中间件系统
- 进行中间件性能对比和分析
- 应用中间件技术解决实际问题
- 验证点：能独立开发异步中间件

In [ ]:
print("\n🔧 异步中间件和后台任务:")
print("=" * 50)

# 1. 自定义异步中间件
print(f"📝 1. 自定义异步中间件:")

# 1.1 请求日志中间件
print(f"\n   📊 1.1 请求日志中间件:")

class RequestLoggingMiddleware:
    """请求日志中间件"""
    
    def __init__(self, app):
        self.app = app
        self.request_logs = []
    
    async def __call__(self, scope, receive, send):
        """中间件调用"""
        if scope["type"] != "http":
            await self.app(scope, receive, send)
            return
        
        # 记录请求开始
        start_time = time.time()
        request_id = str(uuid.uuid4())[:8]
        
        # 获取请求信息
        method = scope["method"]
        path = scope["path"]
        query_string = scope.get("query_string", b"").decode()
        
        # 创建日志条目
        log_entry = {
            "request_id": request_id,
            "method": method,
            "path": path,
            "query": query_string,
            "start_time": start_time,
            "status_code": None,
            "response_time": None
        }
        
        print(f"      🚀 [{request_id}] {method} {path}")
        
        # 包装send函数以捕获响应状态
        async def send_wrapper(message):
            if message["type"] == "http.response.start":
                log_entry["status_code"] = message.get("status", 200)
            await send(message)
        
        try:
            # 调用下一个中间件或路由处理器
            await self.app(scope, receive, send_wrapper)
            
        except Exception as e:
            log_entry["status_code"] = 500
            log_entry["error"] = str(e)
            print(f"      ❌ [{request_id}] 错误: {e}")
            raise
        
        finally:
            # 记录请求完成
            end_time = time.time()
            log_entry["response_time"] = end_time - start_time
            
            self.request_logs.append(log_entry)
            
            print(f"      ✅ [{request_id}] 完成 {log_entry['status_code']} - {log_entry['response_time']:.3f}s")
    
    def get_logs(self, limit: int = 50) -> List[Dict[str, Any]]:
        """获取请求日志"""
        return self.request_logs[-limit:]
    
    def clear_logs(self):
        """清理日志"""
        self.request_logs.clear()

# 1.2 缓存中间件
print(f"\n   💾 1.2 缓存中间件:")

class CacheMiddleware:
    """缓存中间件"""
    
    def __init__(self, app, cache_ttl: int = 300):
        self.app = app
        self.cache_ttl = cache_ttl
        self.cache = {}
        self.cache_stats = {"hits": 0, "misses": 0}
    
    async def __call__(self, scope, receive, send):
        """中间件调用"""
        if scope["type"] != "http":
            await self.app(scope, receive, send)
            return
        
        # 只缓存GET请求
        if scope["method"] != "GET":
            await self.app(scope, receive, send)
            return
        
        # 生成缓存键
        path = scope["path"]
        query_string = scope.get("query_string", b"").decode()
        cache_key = f"{path}?{query_string}"
        
        # 检查缓存
        current_time = time.time()
        
        if cache_key in self.cache:
            cached_data = self.cache[cache_key]
            
            if current_time - cached_data["timestamp"] < self.cache_ttl:
                # 缓存命中
                self.cache_stats["hits"] += 1
                print(f"      🎯 缓存命中: {cache_key}")
                
                # 发送缓存响应
                await send({
                    "type": "http.response.start",
                    "status": cached_data["status"],
                    "headers": cached_data["headers"]
                })
                await send({
                    "type": "http.response.body",
                    "body": cached_data["body"]
                })
                return
            else:
                # 缓存过期，删除
                del self.cache[cache_key]
        
        # 缓存未命中，继续处理请求
        self.cache_stats["misses"] += 1
        
        # 包装send函数以缓存响应
        response_data = {}
        
        async def cache_send(message):
            if message["type"] == "http.response.start":
                response_data["status"] = message.get("status", 200)
                response_data["headers"] = message.get("headers", [])
            elif message["type"] == "http.response.body":
                response_data["body"] = message.get("body", b"")
                
                # 缓存响应（仅缓存成功响应）
                if 200 <= response_data["status"] < 300:
                    self.cache[cache_key] = {
                        "status": response_data["status"],
                        "headers": response_data["headers"],
                        "body": response_data["body"],
                        "timestamp": current_time
                    }
                    print(f"      💾 缓存保存: {cache_key}")
            
            await send(message)
        
        await self.app(scope, receive, cache_send)
    
    def get_cache_stats(self) -> Dict[str, Any]:
        """获取缓存统计"""
        total_requests = self.cache_stats["hits"] + self.cache_stats["misses"]
        hit_rate = self.cache_stats["hits"] / total_requests if total_requests > 0 else 0
        
        return {
            "hits": self.cache_stats["hits"],
            "misses": self.cache_stats["misses"],
            "hit_rate": f"{hit_rate:.1%}",
            "cached_items": len(self.cache),
            "cache_ttl": self.cache_ttl
        }
    
    def clear_cache(self):
        """清理缓存"""
        self.cache.clear()
        self.cache_stats = {"hits": 0, "misses": 0}

# 1.3 安全中间件
print(f"\n   🔒 1.3 安全中间件:")

class SecurityMiddleware:
    """安全中间件"""
    
    def __init__(self, app):
        self.app = app
        self.blocked_ips = set()
        self.suspicious_requests = {}
    
    async def __call__(self, scope, receive, send):
        """中间件调用"""
        if scope["type"] != "http":
            await self.app(scope, receive, send)
            return
        
        # 获取客户端IP
        client_ip = scope.get("client", ["unknown"])[0]
        
        # 检查IP是否被阻止
        if client_ip in self.blocked_ips:
            print(f"      🚫 阻止的IP: {client_ip}")
            await send({
                "type": "http.response.start",
                "status": 403,
                "headers": [(b"content-type", b"application/json")]
            })
            await send({
                "type": "http.response.body",
                "body": json.dumps({"error": "IP被阻止"}).encode()
            })
            return
        
        # 记录可疑请求
        current_time = time.time()
        if client_ip not in self.suspicious_requests:
            self.suspicious_requests[client_ip] = []
        
        # 清理过期记录
        self.suspicious_requests[client_ip] = [
            req_time for req_time in self.suspicious_requests[client_ip]
            if current_time - req_time < 60  # 1分钟内
        ]
        
        # 记录当前请求
        self.suspicious_requests[client_ip].append(current_time)
        
        # 检查请求频率（每分钟超过100次视为可疑）
        if len(self.suspicious_requests[client_ip]) > 100:
            print(f"      ⚠️ 可疑IP: {client_ip}, 请求频率过高")
            
            # 如果频率极高，阻止IP
            if len(self.suspicious_requests[client_ip]) > 200:
                self.blocked_ips.add(client_ip)
                print(f"      🚫 阻止IP: {client_ip}")
                
                await send({
                    "type": "http.response.start",
                    "status": 429,
                    "headers": [(b"content-type", b"application/json")]
                })
                await send({
                    "type": "http.response.body",
                    "body": json.dumps({"error": "请求过于频繁"}).encode()
                })
                return
        
        # 添加安全头
        async def security_send(message):
            if message["type"] == "http.response.start":
                headers = list(message.get("headers", []))
                
                # 添加安全头
                security_headers = [
                    (b"x-content-type-options", b"nosniff"),
                    (b"x-frame-options", b"DENY"),
                    (b"x-xss-protection", b"1; mode=block"),
                    (b"strict-transport-security", b"max-age=31536000; includeSubDomains")
                ]
                
                headers.extend(security_headers)
                message["headers"] = headers
            
            await send(message)
        
        await self.app(scope, receive, security_send)
    
    def get_security_stats(self) -> Dict[str, Any]:
        """获取安全统计"""
        return {
            "blocked_ips": len(self.blocked_ips),
            "suspicious_ips": len(self.suspicious_requests),
            "total_suspicious_requests": sum(
                len(requests) for requests in self.suspicious_requests.values()
            )
        }
    
    def unblock_ip(self, ip: str):
        """解除IP阻止"""
        self.blocked_ips.discard(ip)
        if ip in self.suspicious_requests:
            del self.suspicious_requests[ip]

print(f"   ✅ 自定义异步中间件完成")
print(f"      - RequestLoggingMiddleware: 请求日志记录")
print(f"      - CacheMiddleware: 响应缓存管理")
print(f"      - SecurityMiddleware: 安全防护")

# 2. 后台任务系统
print(f"\n   🔄 2. 后台任务系统:")

class BackgroundTaskManager:
    """后台任务管理器"""
    
    def __init__(self):
        self.task_queue = asyncio.Queue()
        self.running_tasks = {}
        self.completed_tasks = {}
        self.task_stats = {
            "total_tasks": 0,
            "completed_tasks": 0,
            "failed_tasks": 0,
            "running_tasks": 0
        }
        self.worker_running = False
    
    async def add_task(self, task_func, *args, **kwargs):
        """添加后台任务"""
        task_id = str(uuid.uuid4())[:8]
        
        task_info = {
            "id": task_id,
            "func": task_func,
            "args": args,
            "kwargs": kwargs,
            "status": "pending",
            "created_at": time.time(),
            "started_at": None,
            "completed_at": None,
            "error": None
        }
        
        await self.task_queue.put(task_info)
        self.task_stats["total_tasks"] += 1
        
        print(f"      📝 添加后台任务: {task_id}")
        return task_id
    
    async def start_worker(self, max_workers: int = 3):
        """启动工作进程"""
        if self.worker_running:
            return
        
        self.worker_running = True
        
        # 创建多个工作进程
        workers = [
            asyncio.create_task(self._worker(f"worker-{i}"))
            for i in range(max_workers)
        ]
        
        print(f"      🚀 启动 {max_workers} 个后台工作进程")
        return workers
    
    async def _worker(self, worker_name: str):
        """工作进程"""
        print(f"      👷 {worker_name} 开始工作")
        
        while self.worker_running:
            try:
                # 获取任务（超时1秒）
                task_info = await asyncio.wait_for(
                    self.task_queue.get(), 
                    timeout=1.0
                )
                
                # 执行任务
                await self._execute_task(task_info, worker_name)
                
            except asyncio.TimeoutError:
                continue  # 没有任务，继续等待
            except Exception as e:
                print(f"      ❌ {worker_name} 错误: {e}")
        
        print(f"      👷 {worker_name} 停止工作")
    
    async def _execute_task(self, task_info: dict, worker_name: str):
        """执行任务"""
        task_id = task_info["id"]
        
        try:
            # 更新任务状态
            task_info["status"] = "running"
            task_info["started_at"] = time.time()
            self.running_tasks[task_id] = task_info
            self.task_stats["running_tasks"] += 1
            
            print(f"      ⚙️ {worker_name} 开始执行任务: {task_id}")
            
            # 执行任务函数
            result = await task_info["func"](*task_info["args"], **task_info["kwargs"])
            
            # 任务完成
            task_info["status"] = "completed"
            task_info["completed_at"] = time.time()
            task_info["result"] = result
            
            # 移动到完成列表
            del self.running_tasks[task_id]
            self.completed_tasks[task_id] = task_info
            
            self.task_stats["running_tasks"] -= 1
            self.task_stats["completed_tasks"] += 1
            
            execution_time = task_info["completed_at"] - task_info["started_at"]
            print(f"      ✅ {worker_name} 完成任务: {task_id} ({execution_time:.3f}s)")
            
        except Exception as e:
            # 任务失败
            task_info["status"] = "failed"
            task_info["completed_at"] = time.time()
            task_info["error"] = str(e)
            
            # 移动到完成列表
            if task_id in self.running_tasks:
                del self.running_tasks[task_id]
            self.completed_tasks[task_id] = task_info
            
            self.task_stats["running_tasks"] -= 1
            self.task_stats["failed_tasks"] += 1
            
            print(f"      ❌ {worker_name} 任务失败: {task_id} - {e}")
    
    async def stop_worker(self):
        """停止工作进程"""
        self.worker_running = False
        print(f"      🛑 停止后台工作进程")
    
    def get_task_status(self, task_id: str) -> Optional[dict]:
        """获取任务状态"""
        if task_id in self.running_tasks:
            return self.running_tasks[task_id]
        elif task_id in self.completed_tasks:
            return self.completed_tasks[task_id]
        else:
            return None
    
    def get_task_stats(self) -> Dict[str, Any]:
        """获取任务统计"""
        return self.task_stats.copy()
    
    def get_recent_tasks(self, limit: int = 10) -> List[dict]:
        """获取最近的任务"""
        all_tasks = list(self.running_tasks.values()) + list(self.completed_tasks.values())
        
        # 按创建时间排序
        all_tasks.sort(key=lambda x: x["created_at"], reverse=True)
        
        return all_tasks[:limit]

# 创建全局后台任务管理器
background_manager = BackgroundTaskManager()

# 示例后台任务函数
async def send_email_async(to_email: str, subject: str, content: str):
    """异步发送邮件"""
    await asyncio.sleep(random.uniform(2, 5))  # 模拟发送延迟
    
    # 模拟发送结果
    success = random.random() > 0.1  # 90%成功率
    
    if success:
        return {"status": "sent", "to": to_email, "subject": subject}
    else:
        raise Exception(f"发送邮件失败: {to_email}")

async def process_data_async(data_size: int):
    """异步处理数据"""
    await asyncio.sleep(data_size * 0.1)  # 模拟处理时间
    
    # 模拟数据处理结果
    processed_items = data_size * random.randint(10, 50)
    
    return {
        "processed_items": processed_items,
        "data_size": data_size,
        "efficiency": processed_items / data_size
    }

async def generate_report_async(report_type: str):
    """异步生成报告"""
    await asyncio.sleep(random.uniform(3, 8))  # 模拟生成时间
    
    # 模拟报告生成结果
    report_data = {
        "type": report_type,
        "pages": random.randint(5, 20),
        "charts": random.randint(2, 8),
        "file_size": random.randint(100, 500)
    }
    
    return report_data

print(f"   ✅ 后台任务系统完成")
print(f"      - BackgroundTaskManager: 任务队列管理")
print(f"      - 多工作进程支持")
print(f"      - 任务状态跟踪")
print(f"      - 示例任务: 邮件发送、数据处理、报告生成")

# 3. 中间件和后台任务演示
print(f"\n   🎭 3. 中间件和后台任务演示:")

async def demonstrate_middleware_and_tasks():
    """演示中间件和后台任务"""
    print(f"   🔸 中间件和后台任务演示:")
    
    # 启动后台任务工作进程
    workers = await background_manager.start_worker(max_workers=2)
    
    # 添加示例后台任务
    print(f"\n      添加后台任务...")
    
    email_task = await background_manager.add_task(
        send_email_async,
        "user@example.com",
        "测试邮件",
        "这是一封测试邮件内容"
    )
    
    data_task = await background_manager.add_task(
        process_data_async,
        100
    )
    
    report_task = await background_manager.add_task(
        generate_report_async,
        "monthly_report"
    )
    
    # 等待任务执行
    print(f"\n      等待任务执行...")
    await asyncio.sleep(10)
    
    # 检查任务状态
    print(f"\n      任务执行结果:")
    
    for task_id in [email_task, data_task, report_task]:
        task_status = background_manager.get_task_status(task_id)
        if task_status:
            print(f"        任务 {task_id}:")
            print(f"          状态: {task_status['status']}")
            if task_status['status'] == 'completed':
                print(f"          结果: {task_status.get('result', 'N/A')}")
            elif task_status['status'] == 'failed':
                print(f"          错误: {task_status.get('error', 'N/A')}")
            
            if task_status.get('started_at') and task_status.get('completed_at'):
                execution_time = task_status['completed_at'] - task_status['started_at']
                print(f"          执行时间: {execution_time:.3f}s")
        else:
            print(f"        任务 {task_id}: 未找到")
    
    # 显示任务统计
    stats = background_manager.get_task_stats()
    print(f"\n      任务统计:")
    for key, value in stats.items():
        print(f"        {key}: {value}")
    
    # 停止工作进程
    await background_manager.stop_worker()
    
    return "中间件和后台任务演示完成"

# 运行演示
middleware_demo_result = await demonstrate_middleware_and_tasks()
print(f"   {middleware_demo_result}")

print(f"\n✅ 异步中间件和后台任务完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握异步中间件的开发和使用方法")
print(f"   ✓ 理解后台任务的异步处理机制")
print(f"   ✓ 学会请求生命周期管理和错误处理")
print(f"   ✓ 能够实现高效的异步中间件系统")

### 异步Web性能优化 [⭐⭐进阶]
**知识点说明**：异步Web性能优化是构建高性能API服务的关键技术，对于LangChain的大规模部署和并发处理至关重要。

**学习要求**：
- 掌握异步Web性能分析和优化技术
- 理解并发控制和资源管理机制
- 学会异步数据库集成和缓存策略
- 能够实现高性能的异步Web系统

**案例要求**：
- 实现完整的异步Web性能优化系统
- 进行性能对比分析和基准测试
- 应用优化技术解决实际问题
- 验证点：能独立优化异步Web性能

In [ ]:
print("\n⚡ 异步Web性能优化:")
print("=" * 50)

# 1. 性能监控和分析
print(f"📝 1. 性能监控和分析:")

# 1.1 请求性能分析器
print(f"\n   📊 1.1 请求性能分析器:")

@dataclass
class RequestMetrics:
    """请求指标"""
    endpoint: str
    method: str
    status_code: int
    response_time: float
    timestamp: float
    user_id: Optional[str] = None
    error_message: Optional[str] = None

class PerformanceAnalyzer:
    """性能分析器"""
    
    def __init__(self, max_metrics: int = 10000):
        self.metrics = []
        self.max_metrics = max_metrics
        self.real_time_stats = {
            "requests_per_second": 0,
            "average_response_time": 0,
            "error_rate": 0,
            "active_connections": 0
        }
    
    def record_request(self, metrics: RequestMetrics):
        """记录请求指标"""
        self.metrics.append(metrics)
        
        # 限制指标数量
        if len(self.metrics) > self.max_metrics:
            self.metrics = self.metrics[-self.max_metrics:]
        
        # 更新实时统计
        self._update_real_time_stats()
    
    def _update_real_time_stats(self):
        """更新实时统计"""
        if not self.metrics:
            return
        
        # 获取最近1分钟的指标
        current_time = time.time()
        recent_metrics = [
            m for m in self.metrics
            if current_time - m.timestamp < 60
        ]
        
        if recent_metrics:
            # 计算每秒请求数
            self.real_time_stats["requests_per_second"] = len(recent_metrics) / 60
            
            # 计算平均响应时间
            successful_metrics = [m for m in recent_metrics if m.status_code < 400]
            if successful_metrics:
                self.real_time_stats["average_response_time"] = (
                    sum(m.response_time for m in successful_metrics) / 
                    len(successful_metrics)
                )
            
            # 计算错误率
            error_metrics = [m for m in recent_metrics if m.status_code >= 400]
            self.real_time_stats["error_rate"] = len(error_metrics) / len(recent_metrics) * 100
    
    def get_performance_summary(self, time_window: int = 300) -> Dict[str, Any]:
        """获取性能摘要"""
        current_time = time.time()
        window_metrics = [
            m for m in self.metrics
            if current_time - m.timestamp < time_window
        ]
        
        if not window_metrics:
            return {"message": "没有可用的性能数据"}
        
        # 基本统计
        total_requests = len(window_metrics)
        successful_requests = len([m for m in window_metrics if m.status_code < 400])
        error_requests = total_requests - successful_requests
        
        # 响应时间统计
        response_times = [m.response_time for m in window_metrics if m.status_code < 400]
        
        if response_times:
            avg_response_time = sum(response_times) / len(response_times)
            min_response_time = min(response_times)
            max_response_time = max(response_times)
            
            # 计算百分位数
            sorted_times = sorted(response_times)
            p50 = sorted_times[int(len(sorted_times) * 0.5)]
            p95 = sorted_times[int(len(sorted_times) * 0.95)]
            p99 = sorted_times[int(len(sorted_times) * 0.99)]
        else:
            avg_response_time = min_response_time = max_response_time = 0
            p50 = p95 = p99 = 0
        
        # 端点统计
        endpoint_stats = {}
        for metrics in window_metrics:
            endpoint = metrics.endpoint
            if endpoint not in endpoint_stats:
                endpoint_stats[endpoint] = {
                    "requests": 0,
                    "errors": 0,
                    "total_time": 0,
                    "avg_time": 0
                }
            
            endpoint_stats[endpoint]["requests"] += 1
            if metrics.status_code >= 400:
                endpoint_stats[endpoint]["errors"] += 1
            if metrics.status_code < 400:
                endpoint_stats[endpoint]["total_time"] += metrics.response_time
        
        # 计算端点平均时间
        for stats in endpoint_stats.values():
            if stats["requests"] > 0:
                successful_requests = stats["requests"] - stats["errors"]
                if successful_requests > 0:
                    stats["avg_time"] = stats["total_time"] / successful_requests
        
        return {
            "time_window": time_window,
            "total_requests": total_requests,
            "successful_requests": successful_requests,
            "error_requests": error_requests,
            "success_rate": f"{successful_requests/total_requests*100:.1f}%",
            "requests_per_second": total_requests / time_window,
            "response_time": {
                "average": f"{avg_response_time:.3f}s",
                "minimum": f"{min_response_time:.3f}s",
                "maximum": f"{max_response_time:.3f}s",
                "p50": f"{p50:.3f}s",
                "p95": f"{p95:.3f}s",
                "p99": f"{p99:.3f}s"
            },
            "endpoint_stats": endpoint_stats,
            "real_time_stats": self.real_time_stats
        }

# 1.2 并发控制器
print(f"\n   🎛️ 1.2 并发控制器:")

class ConcurrencyController:
    """并发控制器"""
    
    def __init__(self, max_concurrent: int = 100):
        self.max_concurrent = max_concurrent
        self.semaphore = asyncio.Semaphore(max_concurrent)
        self.active_requests = 0
        self.total_requests = 0
        self.rejected_requests = 0
        self.peak_concurrent = 0
    
    async def acquire(self) -> bool:
        """获取并发许可"""
        self.total_requests += 1
        
        try:
            await asyncio.wait_for(self.semaphore.acquire(), timeout=1.0)
            self.active_requests += 1
            self.peak_concurrent = max(self.peak_concurrent, self.active_requests)
            return True
        except asyncio.TimeoutError:
            self.rejected_requests += 1
            return False
    
    def release(self):
        """释放并发许可"""
        if self.active_requests > 0:
            self.active_requests -= 1
            self.semaphore.release()
    
    def get_stats(self) -> Dict[str, Any]:
        """获取并发统计"""
        return {
            "max_concurrent": self.max_concurrent,
            "active_requests": self.active_requests,
            "total_requests": self.total_requests,
            "rejected_requests": self.rejected_requests,
            "peak_concurrent": self.peak_concurrent,
            "rejection_rate": f"{self.rejected_requests/self.total_requests*100:.1f}%" if self.total_requests > 0 else "0%"
        }

# 1.3 异步数据库连接池
print(f"\n   🗄️ 1.3 异步数据库连接池:")

class AsyncDatabasePool:
    """异步数据库连接池"""
    
    def __init__(self, max_connections: int = 20):
        self.max_connections = max_connections
        self.connections = asyncio.Queue(maxsize=max_connections)
        self.active_connections = 0
        self.pool_stats = {
            "total_queries": 0,
            "successful_queries": 0,
            "failed_queries": 0,
            "total_query_time": 0
        }
    
    async def initialize(self):
        """初始化连接池"""
        print(f"      🗄️ 初始化数据库连接池 ({self.max_connections} 连接)")
        
        for i in range(self.max_connections):
            connection = {
                "id": i + 1,
                "created_at": time.time(),
                "last_used": time.time(),
                "query_count": 0
            }
            await self.connections.put(connection)
        
        print(f"      ✅ 数据库连接池初始化完成")
    
    async def get_connection(self) -> dict:
        """获取数据库连接"""
        connection = await asyncio.wait_for(
            self.connections.get(), 
            timeout=5.0
        )
        
        self.active_connections += 1
        connection["last_used"] = time.time()
        
        return connection
    
    async def release_connection(self, connection: dict):
        """释放数据库连接"""
        self.active_connections -= 1
        await self.connections.put(connection)
    
    async def execute_query(self, query: str, params: dict = None) -> dict:
        """执行数据库查询"""
        start_time = time.time()
        
        try:
            # 获取连接
            connection = await self.get_connection()
            
            # 模拟查询执行
            query_time = random.uniform(0.01, 0.1)
            await asyncio.sleep(query_time)
            
            # 更新连接统计
            connection["query_count"] += 1
            
            # 模拟查询结果
            result = {
                "query": query,
                "params": params,
                "rows_affected": random.randint(0, 100),
                "execution_time": query_time,
                "connection_id": connection["id"]
            }
            
            # 更新统计
            self.pool_stats["total_queries"] += 1
            self.pool_stats["successful_queries"] += 1
            self.pool_stats["total_query_time"] += query_time
            
            # 释放连接
            await self.release_connection(connection)
            
            return result
            
        except Exception as e:
            self.pool_stats["total_queries"] += 1
            self.pool_stats["failed_queries"] += 1
            
            if 'connection' in locals():
                await self.release_connection(connection)
            
            raise e
    
    def get_pool_stats(self) -> Dict[str, Any]:
        """获取连接池统计"""
        successful_queries = self.pool_stats["successful_queries"]
        
        if successful_queries > 0:
            avg_query_time = self.pool_stats["total_query_time"] / successful_queries
        else:
            avg_query_time = 0
        
        return {
            "max_connections": self.max_connections,
            "active_connections": self.active_connections,
            "available_connections": self.connections.qsize(),
            "total_queries": self.pool_stats["total_queries"],
            "successful_queries": self.pool_stats["successful_queries"],
            "failed_queries": self.pool_stats["failed_queries"],
            "success_rate": f"{successful_queries/self.pool_stats['total_queries']*100:.1f}%" if self.pool_stats['total_queries'] > 0 else "0%",
            "average_query_time": f"{avg_query_time:.3f}s"
        }

print(f"   ✅ 性能监控和分析完成")
print(f"      - PerformanceAnalyzer: 请求性能分析")
print(f"      - ConcurrencyController: 并发控制管理")
print(f"      - AsyncDatabasePool: 异步数据库连接池")

# 2. 性能优化演示
print(f"\n   🚀 2. 性能优化演示:")

async def demonstrate_performance_optimization():
    """演示性能优化"""
    print(f"   🔸 性能优化演示:")
    
    # 创建性能分析器
    analyzer = PerformanceAnalyzer()
    
    # 创建并发控制器
    concurrency_controller = ConcurrencyController(max_concurrent=10)
    
    # 创建数据库连接池
    db_pool = AsyncDatabasePool(max_connections=5)
    await db_pool.initialize()
    
    # 模拟并发请求
    print(f"\n      模拟50个并发请求...")
    
    async def simulate_request(request_id: int):
        """模拟请求"""
        start_time = time.time()
        
        # 获取并发许可
        if not await concurrency_controller.acquire():
            return {"request_id": request_id, "status": "rejected"}
        
        try:
            # 模拟不同类型的请求
            endpoint = random.choice(["/users", "/messages", "/llm", "/stats"])
            method = random.choice(["GET", "POST", "PUT", "DELETE"])
            
            # 模拟请求处理
            processing_time = random.uniform(0.05, 0.5)
            await asyncio.sleep(processing_time)
            
            # 模拟数据库查询
            if method in ["GET", "POST"]:
                await db_pool.execute_query(
                    f"SELECT * FROM {endpoint.replace('/', '')}",
                    {"request_id": request_id}
                )
            
            end_time = time.time()
            response_time = end_time - start_time
            
            # 模拟响应状态
            status_code = 200 if random.random() > 0.1 else 500
            
            # 记录性能指标
            metrics = RequestMetrics(
                endpoint=endpoint,
                method=method,
                status_code=status_code,
                response_time=response_time,
                timestamp=start_time,
                user_id=f"user_{request_id % 10}",
                error_message=None if status_code == 200 else "Internal Server Error"
            )
            
            analyzer.record_request(metrics)
            
            return {
                "request_id": request_id,
                "status": "completed",
                "response_time": response_time,
                "status_code": status_code
            }
            
        finally:
            concurrency_controller.release()
    
    # 执行并发请求
    start_time = time.time()
    
    tasks = [simulate_request(i) for i in range(50)]
    results = await asyncio.gather(*tasks)
    
    end_time = time.time()
    total_time = end_time - start_time
    
    # 分析结果
    completed_requests = [r for r in results if r["status"] == "completed"]
    rejected_requests = [r for r in results if r["status"] == "rejected"]
    
    print(f"\n      请求执行完成:")
    print(f"        总耗时: {total_time:.3f} 秒")
    print(f"        完成请求: {len(completed_requests)}")
    print(f"        拒绝请求: {len(rejected_requests)}")
    print(f"        吞吐量: {len(completed_requests)/total_time:.1f} 请求/秒")
    
    # 显示性能分析
    performance_summary = analyzer.get_performance_summary(time_window=300)
    print(f"\n      性能分析摘要:")
    print(f"        成功率: {performance_summary['success_rate']}")
    print(f"        平均响应时间: {performance_summary['response_time']['average']}")
    print(f"        P95响应时间: {performance_summary['response_time']['p95']}")
    print(f"        每秒请求数: {performance_summary['requests_per_second']:.1f}")
    
    # 显示并发统计
    concurrency_stats = concurrency_controller.get_stats()
    print(f"\n      并发控制统计:")
    for key, value in concurrency_stats.items():
        print(f"        {key}: {value}")
    
    # 显示数据库连接池统计
    pool_stats = db_pool.get_pool_stats()
    print(f"\n      数据库连接池统计:")
    for key, value in pool_stats.items():
        print(f"        {key}: {value}")
    
    return "性能优化演示完成"

# 运行性能优化演示
performance_demo_result = await demonstrate_performance_optimization()
print(f"   {performance_demo_result}")

print(f"\n✅ 异步Web性能优化完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握异步Web性能分析和优化技术")
print(f"   ✓ 理解并发控制和资源管理机制")
print(f"   ✓ 学会异步数据库集成和缓存策略")
print(f"   ✓ 能够实现高性能的异步Web系统")

## 📋 学习总结

### ✅ 知识清单达成情况验证

**6.5 异步Web框架 [⭐⭐进阶]**
- ✅ 掌握FastAPI异步Web框架的核心概念和使用方法
- ✅ 学会创建异步路由处理器和依赖注入系统
- ✅ 理解Pydantic模型验证和中间件机制
- ✅ 能够构建基础的异步Web应用系统
- ✅ 掌握异步中间件的开发和使用方法
- ✅ 理解后台任务的异步处理机制
- ✅ 学会请求生命周期管理和错误处理
- ✅ 能够实现高效的异步中间件系统
- ✅ 掌握异步Web性能分析和优化技术
- ✅ 理解并发控制和资源管理机制
- ✅ 学会异步数据库集成和缓存策略
- ✅ 能够实现高性能的异步Web系统
- ✅ 能独立构建高性能异步Web应用

### 🎯 与LangChain学习的关联

**异步Web框架重要性**：
- FastAPI异步路由优化LangChain的API响应性能
- 依赖注入模式支持LangChain的组件组合
- 后台任务处理LangChain的长时间推理过程
- 异步Web框架增强LangChain的服务化能力
- 性能优化技术提升LangChain的并发处理能力

**实际应用场景**：
- LangChain的异步API服务和外部接口集成
- LangChain的流式响应和实时处理系统
- LangChain的后台任务队列和批处理系统
- LangChain的高并发Web服务和负载均衡
- LangChain的性能监控和优化系统

### 📚 进阶学习建议

1. **练习建议**：
   - 深入练习FastAPI的高级特性和插件开发
   - 掌握异步WebSocket和Server-Sent Events技术
   - 学习异步微服务架构和服务网格

2. **扩展学习**：
   - 了解异步Web框架的底层实现原理
   - 学习异步Web安全防护和认证授权
   - 探索分布式异步Web系统设计

3. **实际应用**：
   - 构建高性能LangChain API服务
   - 开发实时AI对话和流式响应系统
   - 实现分布式AI模型推理服务

### 🔧 常见错误与注意事项

1. **同步阻塞操作**：
   ```python
   # 错误：在异步路由中使用同步阻塞操作
   @app.post("/process")
   async def process_data(data: DataModel):
       result = some_sync_function()  # 阻塞事件循环
       return result
   
   # 正确：使用异步操作或在线程池中执行同步操作
   @app.post("/process")
   async def process_data(data: DataModel):
       result = await some_async_function()
       return result
   
   # 或者使用线程池
   import concurrent.futures
   
   @app.post("/process")
   async def process_data(data: DataModel):
       loop = asyncio.get_event_loop()
       with concurrent.futures.ThreadPoolExecutor() as pool:
           result = await loop.run_in_executor(pool, some_sync_function)
       return result
   ```

2. **依赖注入滥用**：
   ```python
   # 错误：过度复杂的依赖链
   @app.get("/data")
   async def get_data(
       db: Database = Depends(get_database),
       cache: Cache = Depends(get_cache),
       auth: Auth = Depends(get_auth),
       logger: Logger = Depends(get_logger),
       config: Config = Depends(get_config),
       metrics: Metrics = Depends(get_metrics)
   ):
       # 太多依赖，难以维护和测试
       pass
   
   # 正确：合理使用依赖注入
   @app.get("/data")
   async def get_data(
       db: Database = Depends(get_database),
       current_user: User = Depends(get_current_user)
   ):
       # 只注入必要的依赖
       pass
   ```

3. **后台任务错误处理**：
   ```python
   # 错误：后台任务没有错误处理
   @app.post("/email")
   async def send_email(email: EmailModel, background_tasks: BackgroundTasks):
       background_tasks.add_task(send_email_async, email.to, email.subject)
       # 如果send_email_async失败，错误会被忽略
   
   # 正确：后台任务包含错误处理
   async def send_email_async(to: str, subject: str):
       try:
           # 发送邮件逻辑
           await actual_send_email(to, subject)
           # 记录成功日志
           logger.info(f"邮件发送成功: {to}")
       except Exception as e:
           # 记录错误日志
           logger.error(f"邮件发送失败: {to}, 错误: {e}")
           # 可以考虑重试或通知
           raise e
   ```

4. **中间件顺序不当**：
   ```python
   # 错误：中间件顺序影响功能
   app.add_middleware(ErrorHandlingMiddleware)  # 应该最后
   app.add_middleware(AuthMiddleware)          # 应该在错误处理之前
   app.add_middleware(LoggingMiddleware)       # 应该最先
   
   # 正确：合理的中间件顺序
   app.add_middleware(LoggingMiddleware)       # 最先执行，记录所有请求
   app.add_middleware(AuthMiddleware)          # 认证检查
   app.add_middleware(ErrorHandlingMiddleware) # 最后执行，处理所有错误
   ```

5. **内存泄漏风险**：
   ```python
   # 错误：全局变量不断增长导致内存泄漏
   global_cache = {}
   
   @app.get("/cache/{key}")
   async def get_cache(key: str):
       return global_cache.get(key)
   
   @app.post("/cache/{key}")
   async def set_cache(key: str, value: Any):
       global_cache[key] = value  # 永不清理，内存泄漏
   
   # 正确：使用LRU缓存或定期清理
   from functools import lru_cache
   import time
   
   cache_with_ttl = {}
   
   @app.post("/cache/{key}")
   async def set_cache(key: str, value: Any, ttl: int = 300):
       cache_with_ttl[key] = {
           "value": value,
           "expires_at": time.time() + ttl
       }
   
   # 定期清理过期缓存
   @app.on_event("startup")
   async def startup_event():
       asyncio.create_task(cleanup_expired_cache())
   
   async def cleanup_expired_cache():
       while True:
           current_time = time.time()
           expired_keys = [
               key for key, data in cache_with_ttl.items()
               if current_time > data["expires_at"]
           ]
           for key in expired_keys:
               del cache_with_ttl[key]
           await asyncio.sleep(60)  # 每分钟清理一次
   ```

6. **并发控制不当**：
   ```python
   # 错误：无限制并发可能导致系统崩溃
   @app.post("/heavy-task")
   async def heavy_task():
       # 每个请求都会消耗大量资源
       result = await cpu_intensive_operation()
       return result
   
   # 正确：使用信号量控制并发
   import asyncio
   
   semaphore = asyncio.Semaphore(10)  # 最多10个并发
   
   @app.post("/heavy-task")
   async def heavy_task():
       async with semaphore:
           result = await cpu_intensive_operation()
           return result
   ```

### 🌐 性能优化建议

**路由优化**：
- 使用异步数据库驱动和连接池
- 实现智能缓存策略和失效机制
- 优化序列化和反序列化性能
- 使用适当的HTTP状态码和响应头

**中间件优化**：
- 避免在中间件中执行重操作
- 使用异步日志记录和监控
- 实现请求去重和限流机制
- 优化错误处理和恢复策略

**系统优化**：
- 实现水平扩展和负载均衡
- 使用异步任务队列处理长时间操作
- 监控系统资源使用情况
- 实现优雅关闭和重启机制

---

**🎉 恭喜完成异步Web框架学习！**

你已经全面掌握了异步Web框架的核心技术，能够系统性地进行FastAPI应用开发、异步中间件设计和性能优化，为LangChain智能应用提供了强大的Web服务基础。

## 🚀 下一步学习预告

**第六节：异步编程 - 进行中** 🔄
- 6.1 异步编程基础 ✅
- 6.2 协程与事件循环 ✅
- 6.3 异步I/O操作 ✅
- 6.4 异步网络编程 ✅
- 6.5 异步Web框架 ✅
- 6.6 并发编程模式 ⏳
- 6.7 异步任务队列 ⏳
- 6.8 异步性能优化 ⏳
- 6.9 异步最佳实践 ⏳

**继续第六节：异步编程**
- 下一个：6.6 并发编程模式
- 深入学习并发模式和同步原语
- 掌握生产者消费者模式和线程池

**后续章节预告**：
- Web开发技术
- 项目工程实践

继续加油，异步Web框架已经掌握！准备深入学习并发编程模式！